In [1]:
import os
import json
import requests
import collections.abc
from azureml.core import Workspace, ComputeTarget
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from dotenv import load_dotenv
from azureml.core.compute import AksCompute
from azureml.core.webservice import AksWebservice



C:\Users\thehi\PycharmProjects\pythonProject\venv\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: azureml.core: AzureML support for Python 3.7 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.7 workflows that use AzureML will continue to work without modification, but Python 3.7 users will no longer get access to the latest AzureML features and bugfixes. We recommend that you upgrade to Python 3.8 or newer. To disable SDK V1 deprecation warning set the environment variable AZUREML_DEPRECATE_WARNING to 'False'
  """


In [2]:
# Load the environment variables from .env file
load_dotenv()

# Now you can retrieve each variable using os.getenv('VARIABLE_NAME')
subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
workspace_name = os.getenv('WORKSPACE_NAME')
region = os.getenv('REGION')

# You can now use these variables in your application
print(subscription_id, resource_group, workspace_name, region)


350c509a-aa48-47fd-8252-1342e2ece205 TestGroup WorkSpaceUAE-2 uaenorth


In [3]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Deploying KeyVault with name workspackeyvaultbca5afdd.
Deploying AppInsights with name workspacinsights95d3d1b1.
Deployed AppInsights with name workspacinsights95d3d1b1. Took 12.26 seconds.
Deploying StorageAccount with name workspacstoraged6e2aca3f.
Deployed KeyVault with name workspackeyvaultbca5afdd. Took 29.07 seconds.
Deployed StorageAccount with name workspacstoraged6e2aca3f. Took 29.83 seconds.
Deploying Workspace with name WorkSpaceUAE-2.
Deployed Workspace with name WorkSpaceUAE-2. Took 59.48 seconds.
Workspace WorkSpaceUAE-2 created


In [4]:
# Specify the path to your SavedModel directory
model_path = 'NLP-Project-Sentiment/BuiltModel'
model_name = 'sentiment_analysis_model'

# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path,
                                  model_name=model_name,
                                  workspace=ws)

print("Model registered successfully:", registered_model.id)


Registering model sentiment_analysis_model
Model registered successfully: sentiment_analysis_model:1


In [5]:
# Specify the local file path to the tokenizer
tokenizer_file_path = 'NLP-Project-Sentiment/tokenizer.pickle'  # Update this path

# Specify a name for the tokenizer model
tokenizer_model_name = 'my_tokenizer'

# Register the tokenizer as a model in Azure ML
registered_tokenizer = Model.register(model_path=tokenizer_file_path,
                                      model_name=tokenizer_model_name,
                                      workspace=ws)

print("Tokenizer model registered successfully:", registered_tokenizer.id)


Registering model my_tokenizer
Tokenizer model registered successfully: my_tokenizer:1


In [6]:
# Create a new environment
conda_env = Environment('my-tf-conda-env')
# Define the conda dependencies object
conda_deps = CondaDependencies()
# Add conda and pip packages
# Note: Specify the versions if necessary to ensure compatibility
conda_deps.add_conda_package('numpy')
conda_deps.add_pip_package('tensorflow')  # For TensorFlow

# Set the dependencies for the environment
conda_env.python.conda_dependencies = conda_deps

In [15]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='script.py', environment=conda_env)

In [16]:
inference_config

InferenceConfig(entry_script=script.py, runtime=None, conda_file=None, extra_docker_file_steps=None, source_directory=None, enable_gpu=None, base_image=None, base_image_registry=<azureml.core.container_registry.ContainerRegistry object at 0x000001CF02B51848>)

In [12]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [18]:
# Assuming 'registered_model' is your main model and 'registered_tokenizer' is your tokenizer model
service = Model.deploy(workspace=ws,
                       name='sentiment-analysis-service-2',  # Name of the deployment
                       models=[registered_model, registered_tokenizer],  # Include both models
                       inference_config=inference_config,  # Your inference configuration
                       deployment_config=aci_config)  # Your deployment configuration
service.wait_for_deployment(show_output=True)


C:\Users\thehi\PycharmProjects\pythonProject\venv\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-12 12:49:43+03:00 Creating Container Registry if not exists.
2024-02-12 12:49:50+03:00 Use the existing image.
2024-02-12 12:49:50+03:00 Generating deployment configuration.
2024-02-12 12:49:53+03:00 Submitting deployment to compute..
2024-02-12 12:50:02+03:00 Checking the status of deployment sentiment-analysis-service-2..
2024-02-12 12:52:17+03:00 Checking the status of inference endpoint sentiment-analysis-service-2.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 02769ceb-3b35-4c96-9bee-6fd1b540ed88
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: sentiment-analysis-service-2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 2b6cb3cd62e94ddb8955eb988071702a.azurecr.io/azureml/azureml_36aa3adc3ab2df19e10bc80943b571d8 locally. Please refer to https://aka.ms/debugimage#serv

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 02769ceb-3b35-4c96-9bee-6fd1b540ed88
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: sentiment-analysis-service-2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 2b6cb3cd62e94ddb8955eb988071702a.azurecr.io/azureml/azureml_36aa3adc3ab2df19e10bc80943b571d8 loc

In [14]:
service.get_logs()

'/bin/bash: /azureml-envs/azureml_fa0a7495e1f695c796ee93bc9880efec/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_fa0a7495e1f695c796ee93bc9880efec/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_fa0a7495e1f695c796ee93bc9880efec/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n2024-02-12T09:47:29,833583150+00:00 - rsyslog/run \nbash: /azureml-envs/azureml_fa0a7495e1f695c796ee93bc9880efec/lib/libtinfo.so.6: no version information available (required by bash)\n2024-02-12T09:47:29,840080347+00:00 - gunicorn/run \n2024-02-12T09:47:29,845341207+00:00 | gunicorn/run | \n2024-02-12T09:47:29,846751149+00:00 | gunicorn/run | ###############################################\n2024-02-12T09:47:29,848137091+00:00 | gunicorn/run | AzureML Container Runtime Information\n2024-02-12T09:47:29,856936258+00:00 | gunicorn/run | ##############################

In [ ]:
scoring_uri = service.scoring_uri


In [ ]:
scoring_uri
